# Implementation of LSTM
## Variables to Consider
<ul>
    <li>Hidden Parameters (more seems better until 10)</li>
    <li>Hidden Layers (more than 1 seems useless, makes sense because input is few dimensions)</li>
    <li>Learning Rate (smaller learning rate requires more epoches to achieve similar results)</li>
    <li>Mini-Batch Size (seems to have slight improvement)</li>
    <li>Number Epochs</li>
    <li>Sequence Length (4)</li>
    <li>Length of Prediction (1)</li>
</ul>

In [1]:
#!pip uninstall statsmodels
#!pip install pmdarima
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from tqdm import tqdm
from os import listdir
from torch.autograd import Variable
from sklearn.preprocessing import MinMaxScaler

In [2]:
if torch.cuda.is_available():
    device = torch.device("cuda")
    print("using cuda")

using cuda


In [3]:
class Parameters():
    def __init__(self, num_layers = 1, hidden_size = 10, batch_size = 100, seq_len = 6, pred_len = 3, num_epochs = 20, learning_rate = 0.01, dropout = 1):
        self.num_layers = num_layers
        self.hidden_size = hidden_size
        self.batch_size = batch_size
        self.seq_len = seq_len
        self.pred_len = pred_len
        self.num_epochs = num_epochs
        self.learning_rate = learning_rate
        self.dropout = dropout
    
    def __str__(self):
        return "{}-{}-{}-{}-{}-{}-{}-{}".format(self.num_layers, self.hidden_size, self.batch_size, self.seq_len, self.pred_len, self.num_epochs, self.learning_rate, self.dropout)


Following Code Adapted from ([github sample](https://github.com/spdin/time-series-prediction-lstm-pytorch/blob/master/Time_Series_Prediction_with_LSTM_Using_PyTorch.ipynb))

In [4]:
def readData(file='samples/WY.csv', freq=20, window=6):
    data = pd.read_csv(file, delimiter=',', index_col=0, parse_dates=True)

    #plt.figure(figsize=(20, 4))
    #plt.plot(data)
    #print(data)
    data = data.resample(str(freq) + 'T').mean()
    raw_values = np.asarray(data['CpuUtilizationAverage']).reshape(data.shape[0], 1)
    #plt.plot(data)
    #plt.show()
    print(raw_values)
    return raw_values

training_set = readData()
print(training_set.shape)

[[5.6  ]
 [8.1  ]
 [6.45 ]
 ...
 [9.4  ]
 [5.025]
 [6.125]]
(1080, 1)


In [5]:
from torch.utils.data import TensorDataset, DataLoader

# just used to store some data variables
class Data:
    pass

def large_sliding_windows(data, seq_length, pred_length=2):
    x = []
    y = []

    for i in range(len(data)-seq_length-pred_length+1):
        _x = data[i:(i+seq_length)]
        _y = data[i+seq_length:i+seq_length+pred_length]
        x.append(_x)
        y.append(_y)

    return np.array(x),np.array(y)

def sliding_windows(data, seq_length):
    x = []
    y = []

    for i in range(len(data)-seq_length-1):
        _x = data[i:(i+seq_length)]
        _y = data[i+seq_length]
        x.append(_x)
        y.append(_y)

    return np.array(x),np.array(y)

def generateData(training_set, d, params):
    d.sc = MinMaxScaler()
    training_data = d.sc.fit_transform(training_set) # normalizes the data

    seq_length = params.seq_len # parameter
    pred_len = params.pred_len # parameter
    x, y = large_sliding_windows(training_data, seq_length, pred_len)#sliding_windows(training_data, seq_length)

    batch_size= params.batch_size
    d.max_size = int(batch_size*(len(y)//batch_size))
    d.train_size = int(batch_size*(d.max_size*(0.8)//batch_size)) #train_size = int(len(y) * 0.67)
    d.test_size = d.max_size-d.train_size
    #test_size = len(y) - train_size

    d.dataX = Variable(torch.Tensor(np.array(x[0:d.max_size])))
    d.dataY = Variable(torch.Tensor(np.array(y[0:d.max_size])))

    d.trainX = Variable(torch.Tensor(np.array(x[0:d.train_size])))
    d.trainY = Variable(torch.Tensor(np.array(y[0:d.train_size])))
    train_data = TensorDataset(d.trainX, d.trainY)
    d.train_loader = DataLoader(train_data, shuffle=False, batch_size=batch_size) #primary

    d.testX = Variable(torch.Tensor(np.array(x[d.train_size:d.max_size])))
    d.testY = Variable(torch.Tensor(np.array(y[d.train_size:d.max_size])))
    #test_data = TensorDataset(testX, testY)
    test_data = TensorDataset(d.dataX, d.dataY)
    d.test_loader = DataLoader(test_data, shuffle=False, batch_size=batch_size) #primary
    return d
data = Data()
params = Parameters()
generateData(training_set, data, params)
print(data.trainX.shape)
print(data.trainY.shape)


torch.Size([800, 6, 1])
torch.Size([800, 3, 1])


In [6]:
## Generate Naive Examples

def generateNaive(d, params):
    mean = d.trainY.mean()
    mean_forecasts = np.full(d.testY.shape, mean.item())
    naive_forecasts = np.full(d.testY.shape, d.trainX[-1][-1].item())  # always predicts last provided
    four_naive_forecasts = np.ones(d.testY.shape)
    for row in four_naive_forecasts:
        for i in range(params.pred_len):
            row[i] = d.testX[i][-1].item()
    four_mean_forecasts = np.ones(d.testY.shape)
    for row, test_row in zip(four_mean_forecasts, d.testX):
        for i in range(params.pred_len):
            row[i] = np.mean(test_row.numpy())

    #naive_forecasts = naive_forecasts.reshape(-1, 1)
    #mean_forecasts = mean_forecasts.reshape(-1, 1)

    denormalize=True

    global_naive = naive_forecasts.reshape(naive_forecasts.shape[:-2] + (-1,))
    global_mean = mean_forecasts.reshape(mean_forecasts.shape[:-2] + (-1,))
    local_naive = four_naive_forecasts.reshape(four_naive_forecasts.shape[:-2] + (-1,))
    local_mean = four_mean_forecasts.reshape(four_mean_forecasts.shape[:-2] + (-1,))
    
    if denormalize:
        naive_forecasts = d.sc.inverse_transform(global_naive)
        mean_forecasts = d.sc.inverse_transform(global_mean)
        four_naive_forecasts = d.sc.inverse_transform(local_naive)
        four_mean_forecasts = d.sc.inverse_transform(local_mean)
    
    return (global_naive, global_mean, local_naive, local_mean)

naives  = generateNaive(data, params)
print(data.testY.shape)
print(naives[0].shape)

torch.Size([200, 3, 1])
(200, 3)


In [7]:
class LSTM(nn.Module):
    def __init__(self, output_size, input_size, hidden_dim, n_layers, drop_prob=1):
        super(LSTM, self).__init__()
        self.output_size = output_size
        self.n_layers = n_layers
        self.hidden_dim = hidden_dim
        
        self.lstm = nn.LSTM(input_size, hidden_dim, n_layers, dropout=drop_prob, batch_first=True)
        self.dropout = nn.Dropout(drop_prob)
        self.fc = nn.Linear(hidden_dim, output_size)
        
    def forward(self, x, hidden):
        batch_size = x.size(0)
        #x = x.long()
        lstm_out, hidden = self.lstm(x, hidden)
        #print(lstm_out.shape)
        lstm_out = lstm_out.contiguous().view(-1, self.hidden_dim)
        
        #out = self.dropout(lstm_out)
        #print(lstm_out.shape)
        #out = self.fc(lstm_out) # linear transform from hidden dims to output_size
        #print(out.shape)
        #out = out.view(batch_size, -1) #-1 means size will be infered
        #print(out.shape)
        #out = out[:,-2]
        #print(out.shape)
        #print(hidden[0][-1].shape)
        h_out = hidden[0][-1].view(-1, self.hidden_dim)
        out = self.fc(h_out)
        #print(out.shape)
        return out, hidden
    
    def init_hidden(self, batch_size):
        weight = next(self.parameters()).data
        hidden = (weight.new(self.n_layers, batch_size, self.hidden_dim).zero_().to(device),
                      weight.new(self.n_layers, batch_size, self.hidden_dim).zero_().to(device))
        return hidden

Training

In [8]:
class Model:
    def __init__(self, lstm, h, criterion):
        self.lstm, self.h, self.criterion = lstm, h, criterion

def train(d, params):
    num_epochs = params.num_epochs#30 # seems to stabilize here, higher and it seems to overfit
    learning_rate = params.learning_rate#0.01

    input_size = 1 # required for this time series (value)
    hidden_size = params.hidden_size#2 # hyper parameter
    num_layers = params.num_layers#1 # hyper parameter

    lstm = LSTM(params.pred_len, input_size, hidden_size, num_layers, params.dropout)
    lstm.to(device)

    criterion = torch.nn.MSELoss()    # mean-squared error for regression
    #criterion = torch.nn.BCELoss()
    optimizer = torch.optim.Adam(lstm.parameters(), lr=learning_rate)
    #optimizer = torch.optim.SGD(lstm.parameters(), lr=learning_rate)

    h = lstm.init_hidden(params.batch_size)

    # Train the model
    for epoch in range(num_epochs):
        for x, y in d.train_loader:
            x = x.to(device)
            y = y.to(device)
            h = tuple([e.data for e in h])
            outputs, h = lstm(x, h)
            optimizer.zero_grad() # remove stored gradient

            # obtain the loss function
            #print(outputs.shape)
            y = y.reshape(y.shape[:-2] + (-1,))
            loss = criterion(outputs, y) #.squeeze()
            #print(outputs)

            loss.backward()

            optimizer.step()
        if epoch % 10 == 0:
            print("Epoch: %d, loss: %1.5f" % (epoch, loss.item()))
            #pass
            
    model = Model(lstm, h, criterion)
    return model
            
model = train(data, params)

/home/rayan/.local/lib/python3.8/site-packages/torch/nn/modules/rnn.py:47: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=1 and num_layers=1
  warnings.warn("dropout option adds dropout after all but last "


Epoch: 0, loss: 0.03577
Epoch: 10, loss: 0.00136


Testing

In [24]:
def test(model, d, params, show_plot=False):
    model.lstm.eval()
    test_predict = np.zeros((data.max_size, params.pred_len))
    test_losses = []

    for i, (x, y) in enumerate(d.test_loader):
        model.h = tuple([each.data for each in model.h])
        x, y = x.to(device), y.to(device)
        output, model.h = model.lstm(x, model.h)
        y = y.reshape(y.shape[:-2] + (-1,))
        test_loss = model.criterion(output.squeeze(), y)
        test_losses.append(test_loss.item())
        for j in range(params.batch_size):
            #test_predict.append(output[j].item())
            #test_predict[i*100+j] = output[j].item()
            for k in range(params.pred_len):
                test_predict[j+i*100][k] = output[j][k].item()


    #print(test_predict)

    #data_predict = test_predict.reshape(-1, 1)#test_predict.data.numpy()
    data_predict = test_predict
    dataY_plot = d.dataY.data.numpy()

    dataY_plot = dataY_plot.reshape(dataY_plot.shape[:-2] + (-1,))
    #print(dataY_plot.shape)
    #print(test_predict.shape)

    data_predict_transform = d.sc.inverse_transform(data_predict)
    dataY_plot_transform = d.sc.inverse_transform(dataY_plot)
    
    if show_plot:
        #plt.figure(figsize=(16, 8))
        #plt.axvline(x=d.train_size, c='r', linestyle='--') # data shift from train to test
        #plt.plot(dataY_plot_transform[400:])
        #xpts = [i for i in range(d.train_size-400, dataY_plot_transform.shape[0]-400, 1)]
        #plt.plot(xpts, data_predict_transform[d.train_size:])
        #plt.ylabel("CPU Usage")
        #plt.xlabel("Time")
        #plt.savefig("lstm-out/state-preds/{}.png".format(state), dpi=200, bbox_inches='tight')
        #plt.show()
        
        plt.clf()
        xs = [i for i in range(0, 600, 1)]
        plt.plot(dataY_plot_transform[400:800], c='blue')
        xpts = [i for i in range(d.train_size-400, dataY_plot_transform.shape[0]-400, 1)]
        plt.plot(xpts, data_predict_transform[d.train_size:], c='green')
        plt.plot(xs[400:], dataY_plot_transform[800:], c='blue', alpha=0.3)
        plt.title('CPU Prediction with LSTM')
        plt.xlabel('Time')
        plt.ylabel('CPU Usage')
        #plt.figure(num=None, figsize=(8, 6), dpi=120, facecolor='w', edgecolor='k')
        plt.draw()
        plotname = "lstm-out/state-preds/{}.png".format(state.replace(".csv", ""))
        plt.savefig(plotname, format='png')

    #print(data_predict_transform)
    #print(dataY_plot_transform)
    return (data_predict_transform[d.train_size:], dataY_plot_transform[d.train_size:])

preds, labels = test(model, data, params)


Image is deceptive. Try ~100 samples to see a closer fit

Accuracy Calculations

In [20]:
eps = 1e-5
# Mean Average Percent Error
def mape(preds, labels, prnt=False):
    preds = preds.flatten()
    labels = labels.flatten()
    err = 0
    for i, (pred, label) in enumerate(zip(preds, labels)):
        denum = np.absolute(label) if np.round(label) !=0 else 50#np.max(labels) # this might be wrong
        err += (np.absolute(pred-label) / denum)
            
    err /= preds.shape[0]
    if prnt: print("MAPE - {}".format(err))
    return err
    
# Brier Score or Mean Squared Error
def mse(preds, labels, prnt=False):
    preds = preds.flatten()
    labels = labels.flatten()
    err = np.sum(np.power(preds-labels, 2)) / preds.shape[0]
    if prnt: print("MSE - {}".format(err))
    return err
    
def mae(preds, labels, prnt=False):
    preds = preds.flatten()
    labels = labels.flatten()
    err = (np.sum(np.absolute(preds-labels))) / preds.shape[0]
    if prnt: print("MAE - {}".format(err))
    return err
    
# Root Mean Squared Error
def rmse(preds, labels, prnt=False):
    err = np.power(mse(preds, labels), 0.5)
    if prnt: print("RMSE - {}".format(err))
    return err

# Symmetric Mean Absolute Percentage Error
# some issues in bias, but commonly used
def smape(preds, labels, prnt=False):
    preds = preds.flatten()
    labels = labels.flatten()
    err = 0
    for (pred, label) in zip(preds, labels):
        denum = np.absolute(pred)+np.absolute(label) if np.absolute(pred)+np.absolute(label) !=0 else np.max(labels) #check!!
        err += (np.absolute(pred-label) / denum)
    err /= preds.shape[0] # in textbook, also multiply by 200 but this might be for percentage?
    if prnt: print("SMAPE - {}".format(err))
    return err

#errors = [mape, mse, rmse, smape]
errors = [mae, rmse, mape]

def print_errors(preds, labels, prnt=False):
    err_results = []
    for error in errors:
        err_results.append(error(preds, labels, prnt))
    return err_results
        
print_errors(preds, labels)

for naive in naives:

    print_errors(naive, labels)

ValueError: operands could not be broadcast together with shapes (600,) (200,) 

In [ ]:
"""
params = Parameters()
params.num_epochs=10
params.num_layers=1
params.dropout=1
params.seq_len=6
params.pred_len=3
params.hidden_size=1
"""
ps = []
# num epochs
#ps.append(Parameters(4, 20, 100, 6, 3, 30, 0.01, 0))
#ps.append(Parameters(1, 2, 100, 6, 3, 20, 0.01, 1))
#ps.append(Parameters(1, 5, 100, 6, 1, 30, 0.01, 0.5))
#ps.append(Parameters(1, 2, 100, 6, 3, 40, 0.01, 1))

# batch size
#ps.append(Parameters(1, 2, 50, 6, 3, 30, 0.01, 1))
#ps.append(Parameters(1, 2, 200, 6, 3, 30, 0.01, 1))

# hidden params
#ps.append(Parameters(1, 2, 100, 6, 3, 30, 0.01, 0))
#ps.append(Parameters(1, 5, 100, 6, 3, 30, 0.01, 0))
#ps.append(Parameters(1, 10, 100, 216, 216, 30, 0.01, 0.5))
#ps.append(Parameters(1, 20, 100, 6, 3, 30, 0.01, 0))

# num layers
#ps.append(Parameters(2, 10, 100, 6, 3, 30, 0.01, 0.1))
#ps.append(Parameters(3, 10, 100, 6, 3, 30, 0.01, 0.1))

# dropout (and layers)
#ps.append(Parameters(2, 5, 100, 6, 3, 30, 0.01, 0.9))
#ps.append(Parameters(2, 5, 100, 6, 3, 30, 0.01, 0.5))
#ps.append(Parameters(2, 5, 100, 6, 3, 30, 0.01, 0.1))
#ps.append(Parameters(3, 5, 100, 6, 3, 30, 0.01, 0.9))
#ps.append(Parameters(3, 5, 100, 6, 3, 30, 0.01, 0.5))
#ps.append(Parameters(3, 5, 100, 6, 3, 30, 0.01, 0.1))

# dropout (and layers and epochs)
#ps.append(Parameters(2, 5, 100, 6, 3, 20, 0.01, 0.9))
#ps.append(Parameters(2, 5, 100, 6, 3, 40, 0.01, 0.5))
#ps.append(Parameters(2, 5, 100, 6, 3, 60, 0.01, 0.1))
#ps.append(Parameters(3, 5, 100, 6, 3, 20, 0.01, 0.9))
#ps.append(Parameters(3, 5, 100, 6, 3, 40, 0.01, 0.5))
#ps.append(Parameters(3, 5, 100, 6, 3, 60, 0.01, 0.1))

ps.append(Parameters(1, 30,  100, 432, 216, 30, 0.01, 0))
ps.append(Parameters(1, 5,  100, 432, 216, 30, 0.01, 0))
ps.append(Parameters(1, 10, 100, 432, 216, 30, 0.01, 0))
ps.append(Parameters(1, 20, 100, 432, 216, 30, 0.01, 0))
ps.append(Parameters(1, 30,  100, 432, 216, 30, 0.01, 0.5))
ps.append(Parameters(1, 5,  100, 432, 216, 30, 0.01, 0.5))
ps.append(Parameters(1, 10, 100, 432, 216, 30, 0.01, 0.5))
ps.append(Parameters(1, 20, 100, 432, 216, 30, 0.01, 0.5))
#ps.append(Parameters(1, 2,  100, 432, 216, 30, 0.01, 1))
#ps.append(Parameters(1, 5,  100, 432, 216, 30, 0.01, 1))
#ps.append(Parameters(1, 10, 100, 432, 216, 30, 0.01, 1))
#ps.append(Parameters(1, 20, 100, 432, 216, 30, 0.01, 1))

ps.append(Parameters(2, 30,  100, 432, 216, 30, 0.01, 0))
ps.append(Parameters(2, 5,  100, 432, 216, 30, 0.01, 0))
ps.append(Parameters(2, 10, 100, 432, 216, 30, 0.01, 0))
ps.append(Parameters(2, 20, 100, 432, 216, 30, 0.01, 0))
ps.append(Parameters(2, 30,  100, 432, 216, 30, 0.01, 0.5))
ps.append(Parameters(2, 5,  100, 432, 216, 30, 0.01, 0.5))
ps.append(Parameters(2, 10, 100, 432, 216, 30, 0.01, 0.5))
ps.append(Parameters(2, 20, 100, 432, 216, 30, 0.01, 0.5))
#ps.append(Parameters(2, 2,  100, 432, 216, 30, 0.01, 1))
#ps.append(Parameters(2, 5,  100, 432, 216, 30, 0.01, 1))
#ps.append(Parameters(2, 10, 100, 432, 216, 30, 0.01, 1))
#ps.append(Parameters(2, 20, 100, 432, 216, 30, 0.01, 1))

ps.append(Parameters(3, 30,  100, 432, 216, 30, 0.01, 0))
ps.append(Parameters(3, 5,  100, 432, 216, 30, 0.01, 0))
ps.append(Parameters(3, 10, 100, 432, 216, 30, 0.01, 0))
ps.append(Parameters(3, 20, 100, 432, 216, 30, 0.01, 0))
ps.append(Parameters(3, 30,  100, 432, 216, 30, 0.01, 0.5))
ps.append(Parameters(3, 5,  100, 432, 216, 30, 0.01, 0.5))
ps.append(Parameters(3, 10, 100, 432, 216, 30, 0.01, 0.5))
ps.append(Parameters(3, 20, 100, 432, 216, 30, 0.01, 0.5))
#ps.append(Parameters(3, 2,  100, 432, 216, 30, 0.01, 1))
#ps.append(Parameters(3, 5,  100, 432, 216, 30, 0.01, 1))
#ps.append(Parameters(3, 10, 100, 432, 216, 30, 0.01, 1))
#ps.append(Parameters(3, 20, 100, 432, 216, 30, 0.01, 1))


for params in ps:
    columns = ['MAE', 'RMSE', 'MAPE', 'MEAN_MAE', 'MEAN_RMSE', 'MEAN_MAPE', 'NAIVE_MAE', 'NAIVE_RMSE', 'NAIVE_MAPE']
    df = pd.DataFrame(columns = columns, dtype=np.float64)
    a_row = pd.Series()
    for i, state in enumerate(listdir("samples/")):
        #if state!="OR.csv": continue
        training_set = readData("samples/"+state, freq=20)
        data = Data()
        data.name = state.replace(".csv", "")
        generateData(training_set, data, params)
        #a, b, c, d  = generateNaive(data, params)
        model = train(data, params)
        preds, labels = test(model, data, params)
        #print("----- {} Results -----".format(state.replace(".csv", "")))
        err_results = print_errors(preds, labels)
        #print(a_row.values)
        #err_results.extend(print_errors(d, labels))
        #err_results.extend(print_errors(c, labels))
        err_results.extend([0, 0, 0, 0, 0, 0])
        
        a_row = pd.Series(err_results)
        print(a_row.values)
        row_df = pd.DataFrame([err_results], index = [state.replace(".csv", "")], columns = columns, dtype=np.float64)
        df = pd.concat([row_df, df])
        print(df)
        print("{} - MAE{} - MAPE{}".format(state.replace(".csv", ""), err_results[0], err_results[2]))

    #print(df.shape)
    df.to_csv("lstm-out/{}.csv".format(params))


<ipython-input-32-f40124f90b59>:90: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  a_row = pd.Series()


[[11.325]
 [11.6  ]
 [11.325]
 ...
 [11.4  ]
 [11.2  ]
 [11.45 ]]
Epoch: 0, loss: 0.18683
Epoch: 10, loss: 0.00730
Epoch: 20, loss: 0.00641
[1.41114419 1.95012915 0.10990068 0.         0.         0.
 0.         0.         0.        ]
         MAE      RMSE      MAPE  MEAN_MAE  MEAN_RMSE  MEAN_MAPE  NAIVE_MAE  \
ND  1.411144  1.950129  0.109901       0.0        0.0        0.0        0.0   

    NAIVE_RMSE  NAIVE_MAPE  
ND         0.0         0.0  
ND - MAE1.411144189686135 - MAPE0.10990067618076937
[[7.8  ]
 [8.025]
 [7.475]
 ...
 [7.25 ]
 [6.25 ]
 [6.675]]
Epoch: 0, loss: 0.04546
Epoch: 10, loss: 0.00569
Epoch: 20, loss: 0.00556
[0.88777879 1.43352331 0.1150919  0.         0.         0.
 0.         0.         0.        ]
         MAE      RMSE      MAPE  MEAN_MAE  MEAN_RMSE  MEAN_MAPE  NAIVE_MAE  \
KY  0.887779  1.433523  0.115092       0.0        0.0        0.0        0.0   
ND  1.411144  1.950129  0.109901       0.0        0.0        0.0        0.0   

    NAIVE_RMSE  NAIVE_MAPE  
KY

Epoch: 0, loss: 0.13017
Epoch: 10, loss: 0.04589
Epoch: 20, loss: 0.02412
[4.34157452 5.58972748 1.37133421 0.         0.         0.
 0.         0.         0.        ]
          MAE       RMSE      MAPE  MEAN_MAE  MEAN_RMSE  MEAN_MAPE  NAIVE_MAE  \
ID   4.341575   5.589727  1.371334       0.0        0.0        0.0        0.0   
MA   6.296629   8.507347  1.075468       0.0        0.0        0.0        0.0   
MD  33.699268  39.765747  6.008122       0.0        0.0        0.0        0.0   
NE   6.822412   8.805445  0.362513       0.0        0.0        0.0        0.0   
OK   7.418121   9.509685  2.575074       0.0        0.0        0.0        0.0   
DE   9.581243  11.656006  0.712882       0.0        0.0        0.0        0.0   
CA   7.619375   9.674749  0.531371       0.0        0.0        0.0        0.0   
RI   4.368254   5.364275  1.003006       0.0        0.0        0.0        0.0   
KY   0.887779   1.433523  0.115092       0.0        0.0        0.0        0.0   
ND   1.411144   1.9501

Epoch: 10, loss: 0.10168
Epoch: 20, loss: 0.08779
[20.96909771 26.83180326  2.8345294   0.          0.          0.
  0.          0.          0.        ]
          MAE       RMSE      MAPE  MEAN_MAE  MEAN_RMSE  MEAN_MAPE  NAIVE_MAE  \
GA  20.969098  26.831803  2.834529       0.0        0.0        0.0        0.0   
MO   6.933068   7.373535  1.254051       0.0        0.0        0.0        0.0   
CT  11.265157  14.667587  3.485511       0.0        0.0        0.0        0.0   
TX  20.416892  25.495162  1.232038       0.0        0.0        0.0        0.0   
VA   5.803428   7.813017  4.248761       0.0        0.0        0.0        0.0   
ID   4.341575   5.589727  1.371334       0.0        0.0        0.0        0.0   
MA   6.296629   8.507347  1.075468       0.0        0.0        0.0        0.0   
MD  33.699268  39.765747  6.008122       0.0        0.0        0.0        0.0   
NE   6.822412   8.805445  0.362513       0.0        0.0        0.0        0.0   
OK   7.418121   9.509685  2.575074   

Epoch: 0, loss: 0.01506
Epoch: 10, loss: 0.00832
Epoch: 20, loss: 0.00828
[2.96792771 5.24517395 0.53884451 0.         0.         0.
 0.         0.         0.        ]
          MAE       RMSE      MAPE  MEAN_MAE  MEAN_RMSE  MEAN_MAPE  NAIVE_MAE  \
AL   2.967928   5.245174  0.538845       0.0        0.0        0.0        0.0   
AZ  11.620348  19.065996  3.213344       0.0        0.0        0.0        0.0   
NJ  16.330548  23.577065  2.989853       0.0        0.0        0.0        0.0   
IL   7.854864  11.037736  1.777597       0.0        0.0        0.0        0.0   
GA  20.969098  26.831803  2.834529       0.0        0.0        0.0        0.0   
MO   6.933068   7.373535  1.254051       0.0        0.0        0.0        0.0   
CT  11.265157  14.667587  3.485511       0.0        0.0        0.0        0.0   
TX  20.416892  25.495162  1.232038       0.0        0.0        0.0        0.0   
VA   5.803428   7.813017  4.248761       0.0        0.0        0.0        0.0   
ID   4.341575   5.5897

...

Epoch: 0, loss: 0.25117
Epoch: 10, loss: 0.04781
Epoch: 20, loss: 0.04678
[34.17581597 40.32228832 13.71553941  0.          0.          0.
  0.          0.          0.        ]
          MAE       RMSE       MAPE  MEAN_MAE  MEAN_RMSE  MEAN_MAPE  \
MN  34.175816  40.322288  13.715539       0.0        0.0        0.0   
WY   3.369879   5.066291   0.351847       0.0        0.0        0.0   
ME   6.720446  10.075695   1.060182       0.0        0.0        0.0   
NC  30.725380  36.511927   2.319946       0.0        0.0        0.0   
KS   9.623775  14.159491   1.800215       0.0        0.0        0.0   
NV  39.326545  43.234010   0.447008       0.0        0.0        0.0   
OR  12.702649  19.950272   3.357586       0.0        0.0        0.0   
AK   2.058530   2.693427   0.042807       0.0        0.0        0.0   
TN   7.806168   9.784961   2.617276       0.0        0.0        0.0   
NY   8.471118  10.033272   1.673457       0.0        0.0        0.0   
MI   7.849803   8.478500   1.202098       

Epoch: 10, loss: 0.05511
Epoch: 20, loss: 0.05670
[7.71733891 8.04943656 1.69370851 0.         0.         0.
 0.         0.         0.        ]
          MAE       RMSE       MAPE  MEAN_MAE  MEAN_RMSE  MEAN_MAPE  \
NH   7.717339   8.049437   1.693709       0.0        0.0        0.0   
WA   7.730964  11.114322   6.980745       0.0        0.0        0.0   
VT   5.688915   8.037764   1.849568       0.0        0.0        0.0   
MN  34.175816  40.322288  13.715539       0.0        0.0        0.0   
WY   3.369879   5.066291   0.351847       0.0        0.0        0.0   
ME   6.720446  10.075695   1.060182       0.0        0.0        0.0   
NC  30.725380  36.511927   2.319946       0.0        0.0        0.0   
KS   9.623775  14.159491   1.800215       0.0        0.0        0.0   
NV  39.326545  43.234010   0.447008       0.0        0.0        0.0   
OR  12.702649  19.950272   3.357586       0.0        0.0        0.0   
AK   2.058530   2.693427   0.042807       0.0        0.0        0.0   
TN  

Epoch: 10, loss: 0.09415
Epoch: 20, loss: 0.08111
[8.31329043 9.03157011 3.66853182 0.         0.         0.
 0.         0.         0.        ]
          MAE       RMSE       MAPE  MEAN_MAE  MEAN_RMSE  MEAN_MAPE  \
MT   8.313290   9.031570   3.668532       0.0        0.0        0.0   
CO   7.127040  13.383722   0.996910       0.0        0.0        0.0   
NH   7.717339   8.049437   1.693709       0.0        0.0        0.0   
WA   7.730964  11.114322   6.980745       0.0        0.0        0.0   
VT   5.688915   8.037764   1.849568       0.0        0.0        0.0   
MN  34.175816  40.322288  13.715539       0.0        0.0        0.0   
WY   3.369879   5.066291   0.351847       0.0        0.0        0.0   
ME   6.720446  10.075695   1.060182       0.0        0.0        0.0   
NC  30.725380  36.511927   2.319946       0.0        0.0        0.0   
KS   9.623775  14.159491   1.800215       0.0        0.0        0.0   
NV  39.326545  43.234010   0.447008       0.0        0.0        0.0   
OR  

Epoch: 10, loss: 0.02883
Epoch: 20, loss: 0.02466
[34.86427824 47.60891538 21.67863335  0.          0.          0.
  0.          0.          0.        ]
          MAE       RMSE       MAPE  MEAN_MAE  MEAN_RMSE  MEAN_MAPE  \
SD  34.864278  47.608915  21.678633       0.0        0.0        0.0   
NM   7.664223   9.056241   1.942295       0.0        0.0        0.0   
MT   8.313290   9.031570   3.668532       0.0        0.0        0.0   
CO   7.127040  13.383722   0.996910       0.0        0.0        0.0   
NH   7.717339   8.049437   1.693709       0.0        0.0        0.0   
WA   7.730964  11.114322   6.980745       0.0        0.0        0.0   
VT   5.688915   8.037764   1.849568       0.0        0.0        0.0   
MN  34.175816  40.322288  13.715539       0.0        0.0        0.0   
WY   3.369879   5.066291   0.351847       0.0        0.0        0.0   
ME   6.720446  10.075695   1.060182       0.0        0.0        0.0   
NC  30.725380  36.511927   2.319946       0.0        0.0        0.

Epoch: 0, loss: 0.02510
